# fastbook 05_pet_breeds
> Not done

- toc:true
- branch: master
- badges: false
- comments: false 
- author: 최서연
- categories: [fastbook, pet breeds]

ref: https://github.com/fastai/fastbook

In [1]:
import fastbook
fastbook.setup_book()

In [3]:
from fastbook import *

## Image Classification

- Make them better.
- Apply them to a wider variety of types of data.

## From Dogs and Cats to Pet Breeds

In [4]:
from fastai.vision.all import *
path = untar_data(URLs.PETS)

- Individual files representing items of data, such as text documents or images, possibly organized into folders or with filenames representing information about those items
- A table of data, such as in CSV format, where each row is an item which may include filenames providing a connection between the data in the table and data in other formats, such as text documents and images

 the vast majority of the datasets you'll work with will use some combination of these two formats.

In [7]:
Path.BASE_PATH = path

In [8]:
path.ls()

(#2) [Path('images'),Path('annotations')]

In [10]:
(path/"images").ls()

(#7393) [Path('images/Bombay_13.jpg'),Path('images/beagle_193.jpg'),Path('images/Ragdoll_8.jpg'),Path('images/boxer_106.jpg'),Path('images/keeshond_56.jpg'),Path('images/american_pit_bull_terrier_162.jpg'),Path('images/saint_bernard_136.jpg'),Path('images/staffordshire_bull_terrier_76.jpg'),Path('images/pug_173.jpg'),Path('images/american_pit_bull_terrier_117.jpg')...]